In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import os
import datetime

c_demographics = pd.read_csv('customer_demographics.csv')
c_transaction = pd.read_csv('customer_transaction_data.csv')
item_data = pd.read_csv('item_data.csv')
coupon_item_map = pd.read_csv('coupon_item_mapping.csv')
campaignData = pd.read_csv('campaign_data.csv')
train   = pd.read_csv('train.csv')
test = pd.read_csv('test_QyjYwdj.csv')
sampleSubm = pd.read_csv('sample_submission_Byiv0dS.csv')

item_data['brand_type_category'] = item_data['brand_type'] + '_' + item_data['category']
c_transaction = pd.merge(c_transaction, item_data, on = 'item_id', how = 'left')

c_transaction['date'] = pd.to_datetime(c_transaction['date'], format= '%Y-%m-%d')
campaignData['start_date'] = pd.to_datetime(campaignData['start_date'], format= '%d/%m/%y')
campaignData['end_date'] = pd.to_datetime(campaignData['end_date'], format= '%d/%m/%y')

campaignData = campaignData.sort_values(by = 'start_date')
campaignData['duration'] = campaignData['end_date'] - campaignData['start_date']

from tqdm import tqdm

c_transaction['total_disc'] = c_transaction['other_discount'] + c_transaction.coupon_discount
c_transaction['Total_Bill'] = c_transaction['selling_price'] * c_transaction['quantity'] + c_transaction['total_disc']
c_transaction['dayofweek'] = c_transaction['date'].dt.dayofweek
c_transaction['month'] = c_transaction['date'].dt.month
c_transaction['is_monthEnd'] = c_transaction['date'].dt.day > 25
c_transaction['is_monthStart'] = c_transaction['date'].dt.day < 7
c_transaction['disc_perc'] = abs(c_transaction['other_discount'] + c_transaction['coupon_discount']) / c_transaction['selling_price']
c_transaction['is_coupoun_disc'] = c_transaction['coupon_discount'] < 0
c_transaction['brand_category'] = c_transaction['brand'].astype('str') + '_' + c_transaction['category']
coupon_item_dict = {x : coupon_item_map[coupon_item_map.coupon_id == x]['item_id'].values for x in coupon_item_map.coupon_id.unique()}
c_transaction['dayofmonth'] = c_transaction['date'].dt.day

treat_age = {'70+' : 72, '46-55' : 50.5, '36-45' : 40.5, '26-35' : 30.5, '56-70' : 63, '18-25' : 21.5}
c_demographics['age_approximated'] = c_demographics['age_range'].apply(lambda x : treat_age[x])
c_demographics['income_bucket_div_age'] = c_demographics['income_bracket'] / c_demographics['age_approximated']
treat_family_size = {'1' : 1, '2' : 2, '3' : 3, '4' : 4, '5+' : 5}
c_demographics['family_size'] = c_demographics['family_size'].map(treat_family_size)
c_demographics['income_div_family_size'] = c_demographics['income_bracket'] / c_demographics['family_size']

c_demographics['no_of_children'] = c_demographics['no_of_children'].map({'1' : 1, '2' : 2, '3+' : 3})
c_demographics['income_bucket_div_no_of_child'] = c_demographics['income_bracket'] / c_demographics['no_of_children']

c_demographics['family_to_number_of_child'] = c_demographics['family_size'] / c_demographics['no_of_children']

campaignData['duration'] = campaignData['duration'].dt.days

train = pd.read_pickle('grpd_by_train.pkl')
test = pd.read_pickle('grpd_by_test.pkl')

# pastItem_train = pd.read_csv('item_Past_info_big_imp_TRAIN.csv')
# pastItem_test = pd.read_csv('item_Past_info_big_imp_TEST.csv')
pastItem_train = pd.read_csv('item_Past_info_big_imp_TRAIN_More.csv')
pastItem_test = pd.read_csv('item_Past_info_big_imp_TEST_More.csv')

CouponAggTrain = pd.read_csv('CoupounAggTrain.csv')
CouponAggTest = pd.read_csv('CoupounAggTest.csv')

# CouponAggTest.to_csv('CoupounAggTest.csv', index = False)
# CouponAggTrain.to_csv('CoupounAggTrain.csv', index = False)

CouponAggTrain.rename(columns={'cID' : 'campaign_id', 'coup' : 'coupon_id'}, inplace = True)
CouponAggTest.rename(columns={'cID' : 'campaign_id', 'coup' : 'coupon_id'}, inplace = True)

train = pd.merge(train, pastItem_train, on = ['campaign_id', 'coupon_id', 'customer_id'], how = 'left')
test = pd.merge(test, pastItem_test, on = ['campaign_id', 'coupon_id', 'customer_id'], how = 'left')

train = pd.merge(train, CouponAggTrain, on = ['campaign_id', 'coupon_id'], how = 'left')
test = pd.merge(test, CouponAggTest, on = ['campaign_id', 'coupon_id'], how = 'left')

train = pd.merge(train, c_demographics, on = 'customer_id', how = 'left')
test = pd.merge(test, c_demographics, on = 'customer_id', how = 'left')

train = pd.merge(train, campaignData, on = 'campaign_id', how = 'left')
test = pd.merge(test, campaignData, on = 'campaign_id', how = 'left')

toDrop = [
    'start_date', 
    'end_date', 
]

train.drop(toDrop, axis = 1, inplace=True)
test.drop(toDrop, axis =1, inplace= True)

train.to_pickle('Feat5_119_oct_1_feats_train_More_items_past.pkl')
test.to_pickle('Feat5_119_oct_1_feats_test_More_items_past.pkl')

In [2]:

train = pd.read_pickle('grpd_by_train.pkl')
test = pd.read_pickle('grpd_by_test.pkl')

pastItem_train = pd.read_csv('item_Past_info_big_imp_TRAIN.csv')
pastItem_test = pd.read_csv('item_Past_info_big_imp_TEST.csv')
# pastItem_train = pd.read_csv('item_Past_info_big_imp_TRAIN_More.csv')
# pastItem_test = pd.read_csv('item_Past_info_big_imp_TEST_More.csv')

CouponAggTrain = pd.read_csv('CoupounAggTrain.csv')
CouponAggTest = pd.read_csv('CoupounAggTest.csv')

# CouponAggTest.to_csv('CoupounAggTest.csv', index = False)
# CouponAggTrain.to_csv('CoupounAggTrain.csv', index = False)

CouponAggTrain.rename(columns={'cID' : 'campaign_id', 'coup' : 'coupon_id'}, inplace = True)
CouponAggTest.rename(columns={'cID' : 'campaign_id', 'coup' : 'coupon_id'}, inplace = True)

train = pd.merge(train, pastItem_train, on = ['campaign_id', 'coupon_id', 'customer_id'], how = 'left')
test = pd.merge(test, pastItem_test, on = ['campaign_id', 'coupon_id', 'customer_id'], how = 'left')

train = pd.merge(train, CouponAggTrain, on = ['campaign_id', 'coupon_id'], how = 'left')
test = pd.merge(test, CouponAggTest, on = ['campaign_id', 'coupon_id'], how = 'left')

train = pd.merge(train, c_demographics, on = 'customer_id', how = 'left')
test = pd.merge(test, c_demographics, on = 'customer_id', how = 'left')

train = pd.merge(train, campaignData, on = 'campaign_id', how = 'left')
test = pd.merge(test, campaignData, on = 'campaign_id', how = 'left')

toDrop = [
    'start_date', 
    'end_date', 
]

train.drop(toDrop, axis = 1, inplace=True)
test.drop(toDrop, axis =1, inplace= True)

train.to_pickle('Feat5_119_oct_1_feats_train.pkl')
test.to_pickle('Feat5_119_oct_1_feats_test.pkl')